In [1]:
# Colab: Mesonet API test via static manifest
import requests, pandas as pd, json, io
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots

BASE = "https://d266k7wxhw6o23.cloudfront.net/"
META_MANIFEST_URL = BASE + "metadata/manifest.json"

def url_from_key(key: str) -> str:
    return BASE + key if not key.startswith("http") else key

def get_json(url: str):
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.json()

def get_top_manifest():
    return get_json(META_MANIFEST_URL)

def get_stations_df():
    m = get_top_manifest()
    stations_url = url_from_key(m["stations"]["key"])
    df = pd.DataFrame(get_json(stations_url))
    # normalize numeric columns
    df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
    df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
    return df.dropna(subset=["lat","lon"]).copy()

def get_station_year_manifest(abbrev: str, year: int):
    url = f"{BASE}data/{abbrev}/{year}/manifest.json"
    return get_json(url), url

In [2]:
stations = get_stations_df()
print("Stations loaded:", len(stations))
stations.head(5)

Stations loaded: 88


,id,abbrev,relativeName,lat,lon,elevation,county,timezone,establishedAt,decomission,hasCamera,hasSoil,hasInversion
0,1,FARM,Bowling Green 5 S,36.93,-86.47,559,Warren,US/Central,2007-05-07T00:00:00,None,0,1,1
1,2,RSVL,Russellville 2 W,36.85,-86.92,660,Logan,US/Central,2007-06-27T00:00:00,None,0,1,0
2,3,MRHD,Morehead 4 NE,38.22,-83.48,851,Rowan,US/Eastern,2007-07-12T14:40:00,None,0,1,0
3,4,MRRY,Murray 1 W,36.61,-88.34,568,Calloway,US/Central,2007-11-06T00:00:00,None,0,0,0
4,5,PCWN,Liberty 3 SW,37.28,-84.96,958,Casey,US/Eastern,2007-12-05T00:00:00,None,0,1,0


In [4]:
# ---- Mesonet: pull actual observations (rows) ----
import pandas as pd
from datetime import datetime, timezone, date, timedelta
from typing import Iterable, Tuple, Optional

BASE = "https://d266k7wxhw6o23.cloudfront.net/"
META_MANIFEST_URL = BASE + "metadata/manifest.json"

def url_from_key(key: str) -> str:
    return BASE + key if not key.startswith("http") else key

def get_json(url: str):
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.json()

def get_top_manifest():
    return get_json(META_MANIFEST_URL)

def get_stations_df():
    m = get_top_manifest()
    stations_url = url_from_key(m["stations"]["key"])
    df = pd.DataFrame(get_json(stations_url))
    df["lat"] = pd.to_numeric(df["lat"], errors="coerce")
    df["lon"] = pd.to_numeric(df["lon"], errors="coerce")
    return df.dropna(subset=["lat","lon"]).copy()

def get_variables_df() -> pd.DataFrame:
    """Variables dictionary w/ abbrev, description, units (all metric)."""
    m = get_top_manifest()
    vars_url = url_from_key(m["variables"]["key"])
    vars_json = get_json(vars_url)
    # variables file format is usually a dict keyed by abbrev; make it a DataFrame
    if isinstance(vars_json, dict):
        df = pd.DataFrame(vars_json).T.reset_index().rename(columns={"index":"abbrev"})
    else:
        df = pd.DataFrame(vars_json)
    return df

def get_station_year_manifest(abbrev: str, year: int) -> Tuple[dict, str]:
    url = f"{BASE}data/{abbrev}/{year}/manifest.json"
    return get_json(url), url

# --- drop these in (replace the old _parse_obs_payload_to_df and to_local_time) ---

import re
import numpy as np

DT_RE = re.compile(r"^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$")

def _parse_obs_payload_to_df(payload, *, station_abbrev: str | None = None, stations_df: pd.DataFrame | None = None) -> pd.DataFrame:
    """
    Normalize a station-day JSON into a tidy DataFrame.

    Handles:
      - list of dicts
      - dict with 'data'/'observations'/'obs'/'rows'
      - list of lists (no headers)
    Infers a tz-aware 'time_utc' column by scanning for datetime-like columns.
    Optionally sets 'time_local' if station_abbrev provided.
    """
    # 1) Normalize to a DataFrame of raw values
    if isinstance(payload, list):
        if payload and isinstance(payload[0], dict):
            df = pd.DataFrame(payload)
        elif payload and isinstance(payload[0], (list, tuple)):
            df = pd.DataFrame(payload)  # gives integer columns 0..N-1
        else:
            raise ValueError("Unexpected list payload format.")
    elif isinstance(payload, dict):
        for k in ["data", "observations", "obs", "rows"]:
            if k in payload and isinstance(payload[k], list):
                arr = payload[k]
                if arr and isinstance(arr[0], dict):
                    df = pd.DataFrame(arr)
                else:
                    df = pd.DataFrame(arr)
                break
        else:
            # some providers use {'columns': [...], 'data': [[...],...]}
            if "columns" in payload and "data" in payload and isinstance(payload["data"], list):
                df = pd.DataFrame(payload["data"], columns=payload["columns"])
            else:
                # last resort: flatten dict
                df = pd.json_normalize(payload)
    else:
        raise ValueError("Unsupported payload type.")

    # 2) Find datetime-like columns even if they are unnamed numeric columns
    cand_cols = []
    for c in df.columns:
        s = df[c].astype(str)
        # quick check: does it look like 'YYYY-MM-DD HH:MM:SS' in most rows?
        looks_like_dt = s.str.match(DT_RE).mean() > 0.7
        if looks_like_dt:
            cand_cols.append(c)

    # If that didn't catch them (because dtype isn't string), try to_datetime coercion heuristic
    if not cand_cols:
        for c in df.columns:
            s = pd.to_datetime(df[c], errors="coerce", utc=True)
            if s.notna().mean() > 0.7:
                cand_cols.append(c)

    time_utc = None
    time_local = None

    if cand_cols:
        # If there are multiple time-like cols, try to pick UTC:
        # Heuristic: the column whose values are monotonic AND span close to 24h AND
        # when localized to UTC (as naive->UTC) yields consistent 5/6 hour offset from another.
        # Practically, just prefer the *earliest* times-of-day (UTC tends to be ahead/behind local).
        # We'll try both orders and pick the one with the most non-NaT after parsing.
        parsed = []
        for c in cand_cols:
            t = pd.to_datetime(df[c], errors="coerce", utc=True)
            parsed.append((c, t))

        # choose the candidate with the fewest NaT and most distinct timestamps
        parsed.sort(key=lambda x: (-x[1].notna().sum(), x[1].is_monotonic_increasing is False))

        time_utc = parsed[0][1]
        # If we have a second timestamp column, attempt to treat it as local for convenience
        if len(parsed) >= 2 and station_abbrev is not None:
            # derive local tz from stations metadata
            if stations_df is None:
                stations_df = get_stations_df()
            tz = stations_df.loc[stations_df["abbrev"] == station_abbrev, "timezone"]
            if not tz.empty:
                tzname = tz.iloc[0]
                maybe_local = pd.to_datetime(df[parsed[1][0]], errors="coerce").dt.tz_localize(tzname, nonexistent="shift_forward", ambiguous="NaT")
                time_local = maybe_local
    else:
        # Look for epoch seconds/ms columns
        epoch_col = None
        for c in df.columns:
            if "epoch" in str(c).lower() or "ts" == str(c).lower():
                epoch_col = c; break
        if epoch_col is not None:
            vals = pd.to_numeric(df[epoch_col], errors="coerce")
            # pick seconds vs ms by magnitude
            unit = "ms" if vals.dropna().median() > 1e12 else "s"
            time_utc = pd.to_datetime(vals, errors="coerce", unit=unit, utc=True)

    # 3) Ensure we actually have a tz-aware 'time_utc'
    if time_utc is None or time_utc.isna().all():
        # give user something usable but explicit
        df["time_utc"] = pd.NaT
    else:
        df["time_utc"] = time_utc

    # 4) Optional: attach local time if we have station & tz and didn’t already set it
    if time_local is None and station_abbrev is not None and df["time_utc"].notna().any():
        if stations_df is None:
            stations_df = get_stations_df()
        tz = stations_df.loc[stations_df["abbrev"] == station_abbrev, "timezone"]
        if not tz.empty:
            tzname = tz.iloc[0]
            df["time_local"] = df["time_utc"].dt.tz_convert(tzname)

    # 5) Sort by time_utc when present
    if "time_utc" in df.columns:
        df = df.sort_values("time_utc")

    return df

def to_local_time(df: pd.DataFrame, station_abbrev: str, stations_df: pd.DataFrame | None = None) -> pd.Series:
    """
    Safe local-time converter: first ensure time_utc is tz-aware.
    """
    if "time_utc" not in df.columns or df["time_utc"].isna().all():
        raise TypeError("time_utc is missing or all-NaT; cannot convert. Check the parser and timestamp inference.")
    if stations_df is None:
        stations_df = get_stations_df()
    tz = stations_df.loc[stations_df["abbrev"] == station_abbrev, "timezone"]
    if tz.empty:
        raise ValueError(f"Timezone not found for station {station_abbrev}")
    tzname = tz.iloc[0]
    return df["time_utc"].dt.tz_convert(tzname)

def list_station_dates(abbrev: str, year: int) -> pd.DataFrame:
    """
    Return available YYYYMMDD dates and URLs for a station/year.
    """
    man, _ = get_station_year_manifest(abbrev, year)
    recs = []
    for ymd, meta in man.items():
        recs.append({
            "date": datetime.strptime(ymd, "%Y%m%d").date(),
            "key": meta["key"],
            "url": url_from_key(meta["key"]),
            "file_timestamp": meta.get("timestamp")
        })
    out = pd.DataFrame(recs).sort_values("date")
    out["file_timestamp"] = pd.to_datetime(out["file_timestamp"], errors="coerce")
    return out

def get_station_day_df(abbrev: str, ymd: str | date) -> pd.DataFrame:
    if isinstance(ymd, date):
        ymd_str = ymd.strftime("%Y%m%d")
    else:
        ymd_str = str(ymd)
    year = int(ymd_str[:4])
    man = list_station_dates(abbrev, year)
    row = man.loc[man["date"] == datetime.strptime(ymd_str, "%Y%m%d").date()]
    if row.empty:
        raise ValueError(f"No day file for {abbrev} on {ymd_str}.")
    url = row.iloc[0]["url"]
    payload = get_json(url)
    df = _parse_obs_payload_to_df(payload, station_abbrev=abbrev, stations_df=get_stations_df())
    df["station"] = abbrev
    df["day"] = ymd_str
    return df

def get_station_range_df(abbrev: str, start: str | date, end: Optional[str | date] = None) -> pd.DataFrame:
    if isinstance(start, str): start = datetime.strptime(start, "%Y%m%d").date()
    if end is None: end = start
    if isinstance(end, str): end = datetime.strptime(end, "%Y%m%d").date()
    years = sorted({start.year, end.year})
    catalogs = {y: list_station_dates(abbrev, y) for y in years}
    frames = []
    d = start
    while d <= end:
        cat = catalogs[d.year]
        if (cat["date"] == d).any():
            try:
                frames.append(get_station_day_df(abbrev, d))
            except Exception as e:
                print(f"Warning: failed {abbrev} {d}: {e}")
        else:
            print(f"Missing day for {abbrev} {d}")
        d += timedelta(days=1)
    if not frames:
        raise ValueError("No data found in the requested range.")
    return (pd.concat(frames, ignore_index=True)
              .sort_values("time_utc"))


def to_local_time(df: pd.DataFrame, station_abbrev: str, stations_df: Optional[pd.DataFrame] = None) -> pd.Series:
    """
    Convert df['time_utc'] to the station's local time using its timezone from stations metadata.
    """
    if stations_df is None:
        stations_df = get_stations_df()
    tz = stations_df.loc[stations_df["abbrev"] == station_abbrev, "timezone"]
    if tz.empty:
        raise ValueError(f"Timezone not found for station {station_abbrev}")
    tzname = tz.iloc[0]
    return df["time_utc"].dt.tz_convert(tzname)

# --------- Quick demos ---------

# 1) See what variables exist
variables = get_variables_df()
print("Variables dictionary (first 10):")
display(variables.head(10))

# 2) List all available dates for FARM in 2025 and get the latest day
catalog_2025 = list_station_dates("FARM", 2025)
print("Earliest & latest FARM 2025 dates:", catalog_2025["date"].min(), catalog_2025["date"].max())

latest_day = catalog_2025["date"].max()  # e.g., 2025-11-03 as of the manifest you pasted
print("Latest available day for FARM:", latest_day)

# 3) Load one day of actual observations and peek
farm_latest = get_station_day_df("FARM", latest_day)
print("FARM latest shape:", farm_latest.shape)
display(farm_latest.head())

# 4) Plot a couple of common variables if present (e.g., air temperature 'TAIR', wind speed 'WSPD')
def quick_plot(df: pd.DataFrame, station: str, *vars_to_plot: str):
    import plotly.graph_objects as go
    for v in vars_to_plot:
        if v in df.columns:
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=df["time_utc"], y=df[v], mode="lines", name=v))
            fig.update_layout(
                title=f"{station}: {v} vs time (UTC)",
                xaxis_title="Time (UTC)",
                yaxis_title=v,
                template="plotly_white",
                height=350
            )
            fig.show()
        else:
            print(f"[info] variable '{v}' not found in this day file.")

quick_plot(farm_latest, "FARM", "TAIR", "WSPD")

# 5) Load a multi-day range (e.g., last 3 days) and convert time to the station's local timezone
end_d = catalog_2025["date"].max()
start_d = end_d - timedelta(days=2)
farm_range = get_station_range_df("FARM", start_d, end_d)
farm_range["time_local"] = to_local_time(farm_range, "FARM")
display(farm_range[["time_utc","time_local"] + [c for c in ["TAIR","WSPD","RELH","PRES","RAIN"] if c in farm_range.columns]].head(20))

Variables dictionary (first 10):


,abbrev,label,defaultSoM,units,internalVariable
0,TAIR,Air Temperature,metric,"{'metric': '°C', 'english': '°F'}",False
1,TADP,Dewpoint Depression,metric,"{'metric': '°C', 'english': '°F'}",False
2,DTAR,Change in Temperature,metric,"{'metric': '°C', 'english': '°F'}",False
3,DDWP,Change in Dewpoint,metric,"{'metric': '°C', 'english': '°F'}",False
4,RELH,Relative Humidity,metric,"{'metric': '%', 'english': '%'}",False
5,THMP,Rh Sensor Temperature,metric,"{'metric': '°C', 'english': '°F'}",True
6,DWPT,Dewpoint,metric,"{'metric': '°C', 'english': '°F'}",False
7,WSPD,Wind Speed,metric,"{'metric': 'm/s', 'english': 'mph'}",False
8,WDIR,Wind Direction,metric,"{'metric': 'degrees', 'english': 'degrees'}",False
9,WSMX,Wind Speed at Gust,metric,"{'metric': 'm/s', 'english': 'mph'}",False


Earliest & latest FARM 2025 dates: 2025-01-01 2025-11-03
Latest available day for FARM: 2025-11-03
FARM latest shape: (235, 60)


,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,time_utc,station,day
0,2025-11-03 00:00:00,2025-11-02 18:00:00,2025-11-02 18:00:00,8.53,87.0,0.0,8.40374,8.31570,8.39316,12.52,...,None,9.28,83.0,0.0,0.0,0.0,0.0,2025-11-03 00:00:00+00:00,FARM,20251103
1,2025-11-03 00:05:00,2025-11-02 18:05:00,2025-11-02 18:05:00,8.41,86.5,0.0,8.36618,8.27708,8.35437,12.51,...,None,9.28,82.9,0.0,0.0,0.0,0.0,2025-11-03 00:05:00+00:00,FARM,20251103
2,2025-11-03 00:10:00,2025-11-02 18:10:00,2025-11-02 18:10:00,8.29,87.2,0.0,8.32734,8.23864,8.31581,12.52,...,None,9.29,83.2,0.0,0.0,0.0,0.0,2025-11-03 00:10:00+00:00,FARM,20251103
3,2025-11-03 00:15:00,2025-11-02 18:15:00,2025-11-02 18:15:00,8.25,87.4,0.0,8.12961,8.03663,8.11604,12.52,...,None,9.18,84.4,0.0,0.0,0.0,0.0,2025-11-03 00:15:00+00:00,FARM,20251103
4,2025-11-03 00:20:00,2025-11-02 18:20:00,2025-11-02 18:20:00,8.11,87.2,0.0,7.92495,7.83508,7.90526,12.50,...,None,9.03,84.8,0.0,0.0,0.0,0.0,2025-11-03 00:20:00+00:00,FARM,20251103


[info] variable 'TAIR' not found in this day file.
[info] variable 'WSPD' not found in this day file.


,time_utc,time_local
0,2025-11-01 00:00:00+00:00,2025-10-31 19:00:00-05:00
1,2025-11-01 00:05:00+00:00,2025-10-31 19:05:00-05:00
2,2025-11-01 00:10:00+00:00,2025-10-31 19:10:00-05:00
3,2025-11-01 00:15:00+00:00,2025-10-31 19:15:00-05:00
4,2025-11-01 00:20:00+00:00,2025-10-31 19:20:00-05:00
5,2025-11-01 00:25:00+00:00,2025-10-31 19:25:00-05:00
6,2025-11-01 00:30:00+00:00,2025-10-31 19:30:00-05:00
7,2025-11-01 00:35:00+00:00,2025-10-31 19:35:00-05:00
8,2025-11-01 00:40:00+00:00,2025-10-31 19:40:00-05:00
9,2025-11-01 00:45:00+00:00,2025-10-31 19:45:00-05:00


In [5]:
# ---------- Select variables + plotting helpers ----------

from dataclasses import dataclass

COMMON_VAR_RANGES = {
    "TAIR": (-60, 60),      # °C
    "RELH": (0, 100),       # %
    "WSPD": (0, 60),        # m/s (very generous upper bound)
    "WDIR": (0, 360),       # degrees
    "DWPT": (-60, 60),      # °C
}

@dataclass
class VarSpec:
    abbrev: str
    label: str
    units: str

def _variables_lookup() -> dict:
    """Return {abbrev: VarSpec} from variables metadata."""
    vdf = get_variables_df()
    # Expect columns: abbrev, label, units (dict with 'metric')
    lut = {}
    for _, r in vdf.iterrows():
        units = r.get("units", {})
        metric_unit = None
        if isinstance(units, dict):
            metric_unit = units.get("metric")
        elif isinstance(units, str):
            metric_unit = units
        lut[str(r["abbrev"])] = VarSpec(
            abbrev=str(r["abbrev"]),
            label=str(r.get("label", r["abbrev"])),
            units=str(metric_unit) if metric_unit else ""
        )
    return lut

def preview_numeric_columns(df, n=5):
    """
    Quick glance at unnamed (0..N-1) numeric columns to help manual mapping.
    Shows index, % non-null, min/median/max.
    """
    out = []
    for c in df.columns:
        if isinstance(c, (int, float)) or str(c).isdigit():
            s = pd.to_numeric(df[c], errors="coerce")
            nn = s.notna().mean()
            if nn > 0:  # only show columns that have data
                out.append((c, nn, s.min(), s.median(), s.max()))
    prev = pd.DataFrame(out, columns=["col", "non_null_frac", "min", "median", "max"]).sort_values("col")
    display(prev.head(n if n else len(prev)))
    return prev

def _fits_range(series: pd.Series, lo: float, hi: float, allow_nan_frac=0.5) -> bool:
    s = pd.to_numeric(series, errors="coerce")
    if s.notna().mean() < (1 - allow_nan_frac):
        return False
    s = s.dropna()
    if s.empty:
        return False
    in_range = ((s >= lo) & (s <= hi)).mean()
    # require at least 80% of values inside plausible range
    return in_range > 0.8

def _auto_map_vars(df: pd.DataFrame, want: list[str]) -> dict:
    """
    Heuristic mapper for common vars when day files are list-of-lists.
    Returns {abbrev: column_name_or_index}
    """
    mapping = {}
    numeric_cols = [c for c in df.columns if (isinstance(c, (int, float)) or str(c).isdigit())]
    # Try obvious ones by numeric range
    for var in want:
        rng = COMMON_VAR_RANGES.get(var)
        if not rng:
            continue
        lo, hi = rng
        candidates = []
        for c in numeric_cols:
            s = pd.to_numeric(df[c], errors="coerce")
            if _fits_range(s, lo, hi):
                spread = (s.max() - s.min())
                candidates.append((spread, c))
        # prefer the one with the biggest spread (varies more through the day)
        if candidates:
            candidates.sort(reverse=True)  # by spread
            mapping[var] = candidates[0][1]

    # Small refinement: if both TAIR and DWPT picked the same column, try the 2nd best for DWPT
    if "TAIR" in mapping and "DWPT" in mapping and mapping["TAIR"] == mapping["DWPT"]:
        lo, hi = COMMON_VAR_RANGES["DWPT"]
        alternatives = []
        for c in numeric_cols:
            if c == mapping["TAIR"]:
                continue
            s = pd.to_numeric(df[c], errors="coerce")
            if _fits_range(s, lo, hi):
                alternatives.append(((s.max()-s.min()), c))
        if alternatives:
            alternatives.sort(reverse=True)
            mapping["DWPT"] = alternatives[0][1]
    return mapping

def ensure_vars(df: pd.DataFrame, vars_want: list[str], colmap: dict | None = None, auto=True) -> tuple[pd.DataFrame, dict]:
    """
    Make sure the DataFrame has named columns for each requested variable.
    - If columns already exist (named), we keep them.
    - Else, use 'colmap' (abbrev -> column index or name).
    - Else, if auto=True, guess a mapping for common variables.
    Returns (df_with_columns, mapping_used).
    """
    df = df.copy()
    mapping_used = {}
    # If columns are already named, just pass them through
    todo = []
    for v in vars_want:
        if v in df.columns:
            mapping_used[v] = v
        else:
            todo.append(v)

    # Use explicit mapping first
    if colmap:
        for v in list(todo):
            if v in colmap:
                src = colmap[v]
                if src in df.columns:
                    df[v] = pd.to_numeric(df[src], errors="coerce")
                    mapping_used[v] = src
                    todo.remove(v)
                else:
                    # if source is an int/str index not present, try converting to int
                    try:
                        idx = int(src)
                        if idx in df.columns:
                            df[v] = pd.to_numeric(df[idx], errors="coerce")
                            mapping_used[v] = idx
                            todo.remove(v)
                    except Exception:
                        pass

    # Try auto-mapping for the rest
    if auto and todo:
        auto_map = _auto_map_vars(df, todo)
        for v, src in auto_map.items():
            df[v] = pd.to_numeric(df[src], errors="coerce")
            mapping_used[v] = src
            if v in todo:
                todo.remove(v)

    if todo:
        print(f"[info] Could not map: {', '.join(todo)}. Use 'colmap={{\"VAR\": col_index}}' to specify.")
    return df, mapping_used

def get_station_vars_frame(abbrev: str, start: str | date, end: str | date | None, vars_want: list[str], colmap: dict | None = None, auto=True) -> tuple[pd.DataFrame, dict]:
    """
    Load a station/date range and return a tidy DataFrame with time + requested vars.
    - colmap: optional {abbrev: column_index_or_name} for list-of-lists payloads.
    - auto=True will try to guess columns for common variables.
    Returns (df, mapping_used).
    """
    df = get_station_range_df(abbrev, start, end)
    df, mapping = ensure_vars(df, vars_want, colmap=colmap, auto=auto)
    keep = ["time_utc"]
    if "time_local" in df.columns:
        keep.append("time_local")
    keep.extend([v for v in vars_want if v in df.columns])
    df = df[keep].sort_values("time_utc").reset_index(drop=True)
    return df, mapping

def plot_vars(df: pd.DataFrame, station: str, variables: list[str]):
    import plotly.graph_objects as go
    for v in variables:
        if v not in df.columns:
            print(f"[info] skip {v}: not in dataframe")
            continue
        fig = go.Figure()
        x = df["time_local"] if "time_local" in df.columns else df["time_utc"]
        fig.add_trace(go.Scatter(x=x, y=df[v], mode="lines", name=v))
        unit = ""
        lut = _variables_lookup()
        if v in lut and lut[v].units:
            unit = f" ({lut[v].units})"
        fig.update_layout(
            title=f"{station}: {v}{unit}",
            xaxis_title="Time (local)" if "time_local" in df.columns else "Time (UTC)",
            yaxis_title=f"{v}{unit}",
            template="plotly_white",
            height=350
        )
        fig.show()

In [6]:
# Variables you want
vars_want = ["TAIR", "RELH", "WSPD", "WDIR"]

# Option A — let the code try to auto-map the list-of-lists columns (works well for the 4 above)
df_vars, used = get_station_vars_frame("FARM", "20251101", "20251103", vars_want, colmap=None, auto=True)
print("Column mapping used:", used)  # e.g., {'TAIR': 10, 'RELH': 4, 'WSPD': 7, 'WDIR': 8}
display(df_vars.head())

plot_vars(df_vars, "FARM", vars_want)

# Option B — if you know exact indices, pass them (overrides auto):
# preview = preview_numeric_columns(get_station_day_df("FARM", "20251103"))  # help pick indices
# df_vars, used = get_station_vars_frame("FARM", "20251101", "20251103", vars_want, colmap={"TAIR": 10, "RELH": 4, "WSPD": 7, "WDIR": 8}, auto=False)
# plot_vars(df_vars, "FARM", vars_want)

Column mapping used: {'TAIR': 24, 'RELH': 52, 'WSPD': 24, 'WDIR': 5}


,time_utc,TAIR,RELH,WSPD,WDIR
0,2025-11-01 00:00:00+00:00,0.157,68.56,0.157,0.0
1,2025-11-01 00:05:00+00:00,0.267,66.54,0.267,0.0
2,2025-11-01 00:10:00+00:00,0.247,66.54,0.247,0.0
3,2025-11-01 00:15:00+00:00,0.529,68.94,0.529,0.0
4,2025-11-01 00:20:00+00:00,0.223,70.25,0.223,0.0


In [7]:
# ---------- Map *all* variables (best-effort) + utilities ----------

import numpy as np
import pandas as pd
from dataclasses import dataclass

# Heuristic ranges by unit "type" (metric).
# These are deliberately generous; tighten if needed.
UNIT_RANGES = {
    "°C": (-60, 60),
    "%": (0, 100),
    "m/s": (0, 75),
    "degrees": (0, 360),
    "hPa": (800, 1100),         # sea-level-ish; station PRES may vary
    "kPa": (80, 110),           # if using kPa instead of hPa
    "Pa": (80000, 110000),
    "mm": (0, 500),             # daily precip or accum lines
    "cm": (0, 100),
    "m": (0, 100),              # e.g., snow depth or depth sensors
    "W/m^2": (0, 1400),         # solar radiation
}

# Known variable-specific overrides (if units are ambiguous)
VAR_HINTS = {
    # var: (lo, hi)
    "TAIR": (-60, 60),
    "DWPT": (-60, 60),
    "RELH": (0, 100),
    "WSPD": (0, 75),
    "WSMX": (0, 100),
    "WDIR": (0, 360),
    "PRES": (800, 1100),   # in hPa (common)
    "RAIN": (0, 500),
    "PRCP": (0, 500),
    "SRAD": (0, 1400),
}

@dataclass
class VarMeta:
    abbrev: str
    label: str
    unit_metric: str | None

def _vars_meta() -> dict[str, VarMeta]:
    vdf = get_variables_df()
    out = {}
    for _, r in vdf.iterrows():
        units = r.get("units", {})
        u = None
        if isinstance(units, dict):
            u = units.get("metric")
        elif isinstance(units, str):
            u = units
        out[str(r["abbrev"])] = VarMeta(
            abbrev=str(r["abbrev"]),
            label=str(r.get("label", r["abbrev"])),
            unit_metric=u if u else None
        )
    return out

def _fits_range(series: pd.Series, lo: float, hi: float, min_coverage=0.3, min_inrange=0.7) -> bool:
    s = pd.to_numeric(series, errors="coerce")
    if s.notna().mean() < min_coverage:
        return False
    s = s.dropna()
    if s.empty:
        return False
    frac = ((s >= lo) & (s <= hi)).mean()
    return frac >= min_inrange

def _score_candidate(series: pd.Series) -> float:
    """Higher = 'better': prefer more variance and fewer NaNs."""
    s = pd.to_numeric(series, errors="coerce")
    return s.notna().mean() * (s.max(skipna=True) - s.min(skipna=True))

def infer_all_variables(df_raw: pd.DataFrame, *, prefer_cols_first: bool = True) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Try to produce a DataFrame with time + ALL variables we can map.
    Returns (df_all, mapping_df)
      - df_all has 'time_utc', optional 'time_local', then one column per mapped variable.
      - mapping_df lists: variable, source_column, unit, status ('mapped'/'exists'/'unmapped').
    We keep any *unmapped* numeric columns as 'C{index}' so data isn't lost.
    """
    df = df_raw.copy()

    # Identify time columns
    keep = ["time_utc"]
    if "time_local" in df.columns:
        keep.append("time_local")

    # Everything else we could possibly map from:
    candidate_cols = [c for c in df.columns if c not in keep and not (isinstance(c, str) and c.startswith("time_"))]

    # If the payload already has named variable columns, grab them directly.
    vm = _vars_meta()
    mapped: dict[str, str] = {}
    mapping_rows = []

    # 1) Direct hits (named columns already present)
    for v in vm.keys():
        if v in df.columns:
            mapped[v] = v
            mapping_rows.append({"variable": v, "source_column": v, "unit": vm[v].unit_metric, "status": "exists"})

    # 2) Heuristic mapping for variables not directly present
    todo = [v for v in vm.keys() if v not in mapped]

    # Build list of numeric-only candidate columns (integer-named too)
    num_cands = []
    for c in candidate_cols:
        s = pd.to_numeric(df[c], errors="coerce")
        if s.notna().mean() > 0.1:
            num_cands.append(c)

    # Keep track of which columns we've already consumed
    used_cols: set = set(mapped.values())

    for v in todo:
        # Unit-driven range
        unit = vm[v].unit_metric
        lo, hi = None, None

        if v in VAR_HINTS:
            lo, hi = VAR_HINTS[v]
        elif unit in UNIT_RANGES:
            lo, hi = UNIT_RANGES[unit]

        chosen = None
        if lo is not None:
            # score all candidates that fit the plausible range
            fits = []
            for c in num_cands:
                if c in used_cols:
                    continue
                s = pd.to_numeric(df[c], errors="coerce")
                if _fits_range(s, lo, hi):
                    fits.append(( _score_candidate(s), c ))
            if fits:
                fits.sort(reverse=True)
                chosen = fits[0][1]

        # Light extra hints: wind direction prefers circular-looking values (0..360)
        if chosen is None and v == "WDIR":
            fits = []
            for c in num_cands:
                if c in used_cols: continue
                s = pd.to_numeric(df[c], errors="coerce")
                if s.notna().mean() > 0.4:
                    s2 = s[(s >= 0) & (s <= 360)]
                    if len(s2) / max(1, len(s.dropna())) > 0.9:
                        fits.append((_score_candidate(s2), c))
            if fits:
                fits.sort(reverse=True)
                chosen = fits[0][1]

        if chosen is not None:
            mapped[v] = chosen
            used_cols.add(chosen)
            df[v] = pd.to_numeric(df[chosen], errors="coerce")
            mapping_rows.append({"variable": v, "source_column": chosen, "unit": unit, "status": "mapped"})
        else:
            mapping_rows.append({"variable": v, "source_column": None, "unit": unit, "status": "unmapped"})

    # Build the output frame: time + all mapped variables
    out_cols = keep + sorted([v for v in mapped.keys()])
    out = df[out_cols].copy()

    # Preserve any unmapped numeric columns (so nothing gets dropped)
    unmapped_cols = [c for c in num_cands if c not in used_cols and c not in mapped.keys()]
    # give them stable names
    rename_map = {}
    for c in unmapped_cols:
        cname = f"C{c}" if not isinstance(c, str) else f"C_{c}"
        # avoid collisions
        i = 1
        base = cname
        while cname in out.columns:
            i += 1
            cname = f"{base}_{i}"
        out[cname] = pd.to_numeric(df[c], errors="coerce")
        mapping_rows.append({"variable": cname, "source_column": c, "unit": None, "status": "kept_raw"})

    mapping_df = pd.DataFrame(mapping_rows).sort_values(["status", "variable"]).reset_index(drop=True)
    return out.sort_values("time_utc").reset_index(drop=True), mapping_df

def plot_many(df: pd.DataFrame, station: str, variables: list[str], show_local=True):
    """Plot a bunch of variables, one chart per variable."""
    import plotly.graph_objects as go
    x = df["time_local"] if (show_local and "time_local" in df.columns) else df["time_utc"]
    for v in variables:
        if v not in df.columns:
            print(f"[info] skip {v}: not in dataframe")
            continue
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=x, y=df[v], mode="lines", name=v))
        unit = ""
        vm = _vars_meta()
        if v in vm and vm[v].unit_metric:
            unit = f" ({vm[v].unit_metric})"
        fig.update_layout(
            title=f"{station}: {v}{unit}",
            xaxis_title="Time (local)" if (show_local and "time_local" in df.columns) else "Time (UTC)",
            yaxis_title=f"{v}{unit}",
            template="plotly_white",
            height=300
        )
        fig.show()

In [8]:
# Load a healthy range (e.g., last 3 days) and infer everything we can
df_all_raw = get_station_range_df("FARM", "20251101", "20251103")

df_all, mapping = infer_all_variables(df_all_raw)
print("Mapped columns summary:")
display(mapping)

print("Preview of wide frame (time + all mapped + kept_raw cols):")
display(df_all.head())

# Plot a lot of variables (limit to the ones that actually mapped)
mapped_vars = mapping.query("status in ['exists','mapped']").variable.tolist()
# You can slice to a manageable subset or run all (this will open many charts)
plot_many(df_all, "FARM", mapped_vars[:12])   # first 12 to keep it sane

Mapped columns summary:


,variable,source_column,unit,status
0,C12,12,None,kept_raw
1,C13,13,None,kept_raw
2,C16,16,None,kept_raw
3,C18,18,None,kept_raw
4,C35,35,None,kept_raw
...,...,...,...,...
71,WET1,None,On/Off,unmapped
72,WET2,None,mm/h,unmapped
73,WN20,None,m/s,unmapped
74,WS20,None,m/s,unmapped


Preview of wide frame (time + all mapped + kept_raw cols):


,time_utc,DDWP,DTAR,DWPT,HEAT,PACC,PRCP,PRES,RELH,SM02,...,C18,C35,C36,C37,C38,C39,C40,C41,C42,C_day
0,2025-11-01 00:00:00+00:00,7.758,9.24,9.41940,0.0,78.21,80.90,996.993,0.0,0.063,...,300.0,14.2,0.327,14.2,0.342,14.4,0.35,15.4,0.307,20251101
1,2025-11-01 00:05:00+00:00,7.935,9.00,9.15035,0.0,78.18,80.80,997.013,0.0,0.031,...,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251101
2,2025-11-01 00:10:00+00:00,7.974,8.78,9.08295,0.0,78.14,79.83,997.088,0.0,0.054,...,300.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251101
3,2025-11-01 00:15:00+00:00,7.756,8.60,8.81147,0.0,78.11,79.84,997.119,0.0,0.088,...,299.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251101
4,2025-11-01 00:20:00+00:00,7.592,8.42,8.91379,0.0,78.08,80.50,997.115,0.0,0.089,...,299.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20251101


In [ ]:
# --- Latest DWPT snapshot and spatial map (Plotly) ---

import plotly.express as px
from datetime import timedelta
import numpy as np

def latest_station_day_url(abbrev: str, prefer_year=None):
    """
    Return the URL to the latest available day file for a station.
    If prefer_year is given, try that year first; fall back to scanning nearby years.
    """
    years_try = []
    if prefer_year is None:
        prefer_year = datetime.utcnow().year
    # try prefer_year then a couple around it
    years_try = [prefer_year, prefer_year-1, prefer_year+1]
    best_date = None
    best_url = None
    for y in years_try:
        try:
            cat = list_station_dates(abbrev, y)
        except Exception:
            continue
        if not cat.empty:
            d = cat["date"].max()
            url = cat.loc[cat["date"] == d, "url"].iloc[0]
            if best_date is None or d > best_date:
                best_date, best_url = d, url
    if best_url is None:
        # last resort: just try prefer_year manifest directly
        man, _ = get_station_year_manifest(abbrev, prefer_year)
        if man:
            ymd = max(man.keys())
            best_url = url_from_key(man[ymd]["key"])
            best_date = datetime.strptime(ymd, "%Y%m%d").date()
    return best_date, best_url

# def get_station_latest_dwpt(abbrev: str) -> dict | None:
#     """
#     Fetch station's latest day, infer DWPT, and return its last valid reading.
#     Returns dict with abbrev, time_utc, dwpt_c or None if unavailable.
#     """
#     try:
#         _date, url = latest_station_day_url(abbrev, prefer_year=datetime.utcnow().year)
#         if not url:
#             return None
#         payload = get_json(url)
#         df = _parse_obs_payload_to_df(payload, station_abbrev=abbrev, stations_df=stations)
#         # Make sure we have DWPT (infer if needed)
#         df2, mapping = ensure_vars(df, ["DWPT"], colmap=None, auto=True)
#         if "DWPT" not in df2.columns:
#             return None
#         # take the last non-NaN dewpoint
#         s = df2.dropna(subset=["DWPT"]).sort_values("time_utc").tail(1)
#         if s.empty:
#             return None
#         row = s.iloc[0]
#         return {
#             "abbrev": abbrev,
#             "time_utc": row["time_utc"],
#             "dwpt_c": float(row["DWPT"])
#         }
#     except Exception as e:
#         # print(f"skip {abbrev}: {e}")
#         return None

def get_station_latest_dwpt(abbrev: str) -> dict | None:
    """
    Fetch station's latest day, infer DWPT, and return its last reading,
    with impossible values (>200 °C) replaced by NaN.
    Returns dict with abbrev, time_utc, dwpt_c (may be NaN) or None if no time found.
    """
    try:
        _date, url = latest_station_day_url(abbrev, prefer_year=datetime.utcnow().year)
        if not url:
            return None
        payload = get_json(url)
        df = _parse_obs_payload_to_df(payload, station_abbrev=abbrev, stations_df=stations)

        # Ensure DWPT exists (create via auto-mapping if needed)
        df2, _ = ensure_vars(df, ["DWPT"], colmap=None, auto=True)
        if "DWPT" not in df2.columns:
            # no dewpoint column at all; still return the timestamp for alignment if present
            last = df2.sort_values("time_utc").tail(1)
            if last.empty or "time_utc" not in last.columns or pd.isna(last.iloc[0]["time_utc"]):
                return None
            return {
                "abbrev": abbrev,
                "time_utc": last.iloc[0]["time_utc"],
                "dwpt_c": float("nan")
            }

        # Take the latest row (even if DWPT is NaN)
        last = df2.sort_values("time_utc").tail(1)
        if last.empty or "time_utc" not in last.columns or pd.isna(last.iloc[0]["time_utc"]):
            return None

        # Coerce to numeric and apply QC: >200 °C -> NaN
        val = pd.to_numeric(last.iloc[0]["DWPT"], errors="coerce")
        if pd.notna(val) and val > 200:
            val = float("nan")

        return {
            "abbrev": abbrev,
            "time_utc": last.iloc[0]["time_utc"],
            "dwpt_c": float(val) if pd.notna(val) else float("nan")
        }
    except Exception:
        return None

def latest_dwpt_snapshot(window_minutes: int = 15) -> pd.DataFrame:
    """
    Collect latest DWPT for all stations and align to a network 'latest' timestamp.
    Keeps stations whose latest reading is within ±window_minutes of the network max timestamp.
    """
    recs = []
    for ab in stations["abbrev"].tolist():
        out = get_station_latest_dwpt(ab)
        if out:
            recs.append(out)
    snap = pd.DataFrame(recs)
    if snap.empty:
        raise RuntimeError("No DWPT readings found across stations.")
    # global latest timestamp
    tmax = snap["time_utc"].max()
    window = pd.Timedelta(minutes=window_minutes)
    keep = (snap["time_utc"] >= tmax - window) & (snap["time_utc"] <= tmax + window)
    snap = snap.loc[keep].copy()

    # join lat/lon/name
    cols = ["abbrev", "relativeName", "lat", "lon", "county", "elevation", "timezone"]
    meta = stations[["abbrev","relativeName","lat","lon","county","elevation","timezone"]]
    out = snap.merge(meta, on="abbrev", how="left")
    out["asof_utc"] = tmax
    return out

def plot_dwpt_map(snapshot_df: pd.DataFrame):
    """
    Plot DWPT (°C) on a map. Handles NaNs safely for size and color.
    """
    df = snapshot_df.copy()

    # Coerce & QC here too (belt-and-suspenders)
    df["dwpt_c"] = pd.to_numeric(df["dwpt_c"], errors="coerce").where(lambda s: s <= 200)

    df["name"] = df["relativeName"] + " (" + df["abbrev"] + ")"
    # nice hover with NaN shown as "—"
    dwpt_str = df["dwpt_c"].round(2).astype("Float64").astype(str).str.replace("<NA>", "—")
    df["hover"] = (
        df["name"]
        + "<br>Dewpoint: " + dwpt_str + " °C"
        + "<br>As of (UTC): " + df["time_utc"].dt.strftime("%Y-%m-%d %H:%M:%S")
    )

    # --- SAFE SIZE: constant fallback when all NaN; scaled otherwise ---
    s = pd.to_numeric(df["dwpt_c"], errors="coerce")
    if s.notna().any():
        smin, smax = s.min(), s.max()
        rng = (smax - smin) if pd.notna(smin) and pd.notna(smax) and (smax > smin) else 1.0
        size = 8 + 6 * ((s - smin) / rng)
    else:
        size = pd.Series(8, index=df.index)
    size = size.fillna(8).clip(6, 14)

    # Only color by dwpt if we have at least one non-NaN; else no color scale
    color_arg = "dwpt_c" if df["dwpt_c"].notna().any() else None

    center = {"lat": float(df["lat"].mean()), "lon": float(df["lon"].mean())}
    fig = px.scatter_mapbox(
        df,
        lat="lat", lon="lon",
        color=color_arg,
        color_continuous_scale="Turbo" if color_arg else None,
        hover_name="name",
        hover_data={"dwpt_c": True, "county": True, "elevation": True, "timezone": True, "time_utc": True},
        size=size,
        height=650, zoom=6
    )
    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox_center=center,
        margin=dict(l=0, r=0, t=40, b=0),
        coloraxis_colorbar=dict(title="Dewpoint (°C)") if color_arg else None,
        title=f"Mesonet Dewpoint Snapshot · as of {df['asof_utc'].iloc[0].strftime('%Y-%m-%d %H:%M UTC')}"
    )
    fig.show()

# ---- Run it
stations = get_stations_df()  # ensure we have station metadata
snap = latest_dwpt_snapshot(window_minutes=15)   # tighten/loosen the window as needed
print(f"{len(snap)} stations within ±15 min of network latest {snap['asof_utc'].iloc[0]}")
display(snap[["abbrev","relativeName","time_utc","dwpt_c","lat","lon"]].sort_values("abbrev").head())

plot_dwpt_map(snap)

/tmp/ipython-input-4008837750.py:74: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

/tmp/ipython-input-4008837750.py:74: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

/tmp/ipython-input-4008837750.py:74: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

/tmp/ipython-input-4008837750.py:74: DeprecationWarning:

datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).

/tmp/ipython-input-4008837750.py